# PRACTISE

In [9]:
import asyncio
import aiohttp
import time
from typing import List

## 1. BASIC ASYNC FUNCTION

In [10]:
async def simple_async_function():
    """A basic async function that simulates some work"""
    print("Starting async work...")
    await asyncio.sleep(2)  # Simulates I/O operation (non-blocking)
    print("Async work completed!")
    return "Task finished"

# await simple_async_function()

## 2. COMPARE ASYNC AND SYNC

In [15]:
def synchronous_task(task_id: int, duration: int) -> str:
    """Synchronous version - blocks the thread"""
    print(f"Sync Task {task_id} started")
    time.sleep(duration)  # This BLOCKS the thread
    print(f"Sync Task {task_id} completed")
    return f"Sync result {task_id}"

synchronous_task(1, 5)

Sync Task 1 started
Sync Task 1 completed


'Sync result 1'

In [17]:
async def asynchronous_task(task_id: int, duration: int) -> str:
    """Asynchronous version - doesn't block"""
    print(f"Async Task {task_id} started")
    await asyncio.sleep(duration)  # This DOESN'T block the thread
    print(f"Async Task {task_id} completed")
    return f"Async result {task_id}"

await asynchronous_task(1,5)

Async Task 1 started
Async Task 1 completed


'Async result 1'

## 3. RUNNING MULTIPLE ASYNC TASKS

In [18]:
async def run_sequential_tasks():
    """Run tasks one after another (slower)"""
    print("\n--- SEQUENTIAL EXECUTION ---")
    start_time = time.time()

    result1 = await asynchronous_task(1,2)
    result2 = await asynchronous_task(2,2)
    result3 = await asynchronous_task(3,2)

    end_time = time.time()
    print(f"Sequential time: {end_time - start_time:.2f} seconds")
    return [result1, result2, result3]

await run_sequential_tasks()


--- SEQUENTIAL EXECUTION ---
Async Task 1 started
Async Task 1 completed
Async Task 2 started
Async Task 2 completed
Async Task 3 started
Async Task 3 completed
Sequential time: 6.00 seconds


['Async result 1', 'Async result 2', 'Async result 3']

In [19]:
async def run_concurrent_tasks():
    """Run tasks concurrently (faster)"""
    print("\n--- CONCURRENT EXECUTION ---")
    start_time = time.time()

    # Create tasks but don't await them yet
    tasks = [
        asynchronous_task(1, 2),
        asynchronous_task(2, 2),
        asynchronous_task(3, 2)
    ]

    # Wait for all tasks to complete concurrently
    results = await asyncio.gather(*tasks)

    end_time = time.time()
    print(f"Concurrent time: {end_time - start_time:.2f} seconds")
    return results

await run_concurrent_tasks()


--- CONCURRENT EXECUTION ---
Async Task 1 started
Async Task 2 started
Async Task 3 started
Async Task 1 completed
Async Task 2 completed
Async Task 3 completed
Concurrent time: 2.00 seconds


['Async result 1', 'Async result 2', 'Async result 3']

## 4. REAL-WORLD EXAMPLE: HTTP REQUESTS

In [22]:
async def fetch_url(session: aiohttp.ClientSession, url: str) -> dict:
    """Fetch a single URL asynchronously"""
    try:
        print(f"Fetching {url}...")
        async with session.get(url) as response:
            data = await response.json()
            print(f"✓ Fetched {url} - Status: {response.status}")
            return {"url": url, "status": response.status, "data": data}
    except Exception as e:
        print(f"✗ Error fetching {url}: {e}")
        return {"url": url, "error": str(e)}


url = "https://jsonplaceholder.typicode.com/posts/1"
# FIX: Corrected typo from ClientSEssion to ClientSession
async with aiohttp.ClientSession() as session:
    result = await fetch_url(session, url)
    print(result)

Fetching https://jsonplaceholder.typicode.com/posts/1...
✓ Fetched https://jsonplaceholder.typicode.com/posts/1 - Status: 200
{'url': 'https://jsonplaceholder.typicode.com/posts/1', 'status': 200, 'data': {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}}


In [23]:
async def fetch_multiple_urls(urls: List[str]) -> List[dict]:
    """Fetch multiple URLs concurrently"""
    print("\n--- FETCHING URLS CONCURRENTLY ---")
    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        # Create tasks for all URLs
        tasks = [fetch_url(session, url) for url in urls]

        # Wait for all requests to complete
        results = await asyncio.gather(*tasks, return_exceptions=True)

    end_time = time.time()
    print(f"All requests completed in {end_time - start_time:.2f} seconds")
    return results

urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/posts/2",
    "https://jsonplaceholder.typicode.com/posts/3"
]

results = await fetch_multiple_urls(urls)
for result in results:
    print(result, end="\n")


--- FETCHING URLS CONCURRENTLY ---
Fetching https://jsonplaceholder.typicode.com/posts/1...
Fetching https://jsonplaceholder.typicode.com/posts/2...
Fetching https://jsonplaceholder.typicode.com/posts/3...
✓ Fetched https://jsonplaceholder.typicode.com/posts/2 - Status: 200
✓ Fetched https://jsonplaceholder.typicode.com/posts/3 - Status: 200
✓ Fetched https://jsonplaceholder.typicode.com/posts/1 - Status: 200
All requests completed in 0.33 seconds
{'url': 'https://jsonplaceholder.typicode.com/posts/1', 'status': 200, 'data': {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}}
{'url': 'https://jsonplaceholder.typicode.com/posts/2', 'status': 200, 'data': {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil repreh

## 5. ASYNC CONTEXT MANAGERS AND GENERATORS

In [ ]:
class AsyncResourceManager:
    """Example of async context manager"""

    async def __aenter__(self):
        print("Acquiring async resource...")
        await asyncio.sleep(0.5)  # Simulate resource acquisition
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print("Releasing async resource...")
        await asyncio.sleep(0.5)  # Simulate resource cleanup

In [27]:
async def use_async_context_manager():
    """Demonstrate async context manager usage"""
    print("\n--- ASYNC CONTEXT MANAGER ---")
    async with AsyncResourceManager() as resource:
        print("Using the async resource...")
        await asyncio.sleep(1)
    print("Resource automatically cleaned up!")


await use_async_context_manager()


--- ASYNC CONTEXT MANAGER ---
Acquiring async resource...
Using the async resource...
Releasing async resource...
Resource automatically cleaned up!


## 6. ERROR HANDLING IN ASYNC CODE


In [ ]:
async def task_that_might_fail(task_id: int, should_fail: bool = False):
    """Task that might raise an exception"""
    await asyncio.sleep(1)
    if should_fail:
        raise ValueError(f"Task {task_id} failed!")
    return f"Task {task_id} succeeded"

async def handle_async_errors():
    """Demonstrate error handling in async code"""
    print("\n--- ERROR HANDLING ---")

    tasks = [
        task_that_might_fail(1, False),
        task_that_might_fail(2, True),   # This will fail
        task_that_might_fail(3, False),
    ]

    # Method 1: Handle each task individually
    results = []
    for task in tasks:
        try:
            result = await task
            results.append(result)
            print(f"✓ {result}")
        except Exception as e:
            error_msg = f"✗ Error: {e}"
            results.append(error_msg)
            print(error_msg)

    return results

await handle_async_errors()


--- ERROR HANDLING ---
[<coroutine object task_that_might_fail at 0x10b200e40>, <coroutine object task_that_might_fail at 0x10b59f060>, <coroutine object task_that_might_fail at 0x10b59f760>]
✓ Task 1 succeeded
✗ Error: Task 2 failed!
✓ Task 3 succeeded


['Task 1 succeeded', '✗ Error: Task 2 failed!', 'Task 3 succeeded']

## 7. MAIN EXECUTION FUNCTION


## 8. HOW TO RUN THIS CODE